In [24]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np

spy = pd.read_csv("SPY_Sim.csv")
spy = spy.set_index("Date")
#spy

In [90]:
VAL_TARGET = 2203.3810409
CATCHUP_INIT = 5000
SICNE_PEAK_TIME_MON = 15

def mysum(lst):
    sum = 0
    for one in lst:
        sum += one if isinstance(one, (int, float)) else 0
    return sum

def avg(lst):
    return sum(lst) / len(lst)

valTarget, targetCumRequired, ntbStockCost = 'Value Target $', 'Target.Cum.Required $', 'NTB Stock Cost $'
ColListZeroDollar = [valTarget, targetCumRequired, ntbStockCost]
for one in ColListZeroDollar:
    spy[one] = 0.0

inStockNum, needToBuyStockNum, adjNTBStockNum = 'In-Stock #', 'NeedToBuyStock #', 'Adj.NTB Stock #'
inventoryStockNum  = 'InventoryStock #'
ColListZeroInt = [inStockNum, needToBuyStockNum, adjNTBStockNum, inventoryStockNum]
for one in ColListZeroInt:
    spy[one] = 0

cumDesc, pLessMA12, peakTime = 'Cum.Desc', 'P<MA12', 'PeakTime'
sincePeakTime, avgTop5MinInLst36m, diffCurVsTop5min = 'sincePeakTime', 'AvgTop5Min$inlst36' ,'DiffCurVsTop5min'
catchUpPrice, catchUpBuy = 'CatchUpPrice', 'CatchUpBuy'
ColListEmpty = [cumDesc, pLessMA12, peakTime, sincePeakTime, avgTop5MinInLst36m, diffCurVsTop5min,
          catchUpPrice, catchUpBuy]
for one in ColListEmpty:
    spy[one] = ''
#spy
#tmp = avg(spy.iloc[0:2]['Close'])
#tmp
lst = ['', '', 1, 1.2]
print(mysum(lst))

2.2


In [112]:
def setVal(df, idx, colname, val):
    df.iloc[idx, df.columns.get_loc(colname)] = val
    
def calCumDesc(df, idx):
    # close < average last six month and drop more than 1% than last month
    isTrue = idx >= 6 and df.iloc[idx]['Close'] < avg(df.iloc[idx-6:idx]['Close'])
    isTrue = isTrue and df.iloc[idx]['Close']/df.iloc[idx-1]['Close'] - 1 < -0.01
    isTrue and setVal(df, idx, cumDesc, 1)
    #setVal(df, idx, cumDesc, 1 if isTrue else "")

def calPLessMA12(df, idx):
    # close/MA12 - 1 < -10% (eqa) and sum(CumDesc of last 3 month) > 1 : return (eqa)
    if idx >= 12:
        p = df.iloc[idx]['Close']/avg(df.iloc[idx-12:idx]['Close']) - 1                                      
        isTrue = p < -0.1 and mysum(df.iloc[idx-3:idx][cumDesc]) > 1
        isTrue and setVal(df, idx, pLessMA12, p)
                                                     
def calPeakTime(df, idx):
    # if pLessMA12 < -10%, return history high idx, return -1
    if df.iloc[idx][pLessMA12]:
        pos = np.argmax(df.iloc[0:idx]['Close'])
        setVal(df, idx, peakTime, pos)

def calSincePeakTime(df, idx):
    # if PeakTime >= 0, return idx-PeakTime
    if df.iloc[idx][peakTime]:
        setVal(df, idx, sincePeakTime, idx - df.index.get_loc(df.iloc[idx][peakTime]))

def calAvgTop5MinInLst36m(df, idx):
    # if SincepeakTime no Null, return average(top 5 min with in last 36)
    if df.iloc[idx][sincePeakTime] and idx >= 37:
        last36 = list(df.iloc[idx-37:idx]['Close'])
        last36.sort()
        setVal(df, idx, avgTop5MinInLst36m, avg(last36[:5]))
        #print("%s %s" % (df.index[idx], avg(last36[:5])))

def calDiffCurVsTop5min(df, idx):
    # if AvgTop5MinInLst36m not null: return close/ AvgTop5MinInLst36m -1
    if df.iloc[idx][avgTop5MinInLst36m]:
        p = df.iloc[idx]['Close']/df.iloc[idx][avgTop5MinInLst36m] - 1
        setVal(df, idx, diffCurVsTop5min, p)

def calCatchUpPrice(df, idx):
    # if SincePeakTime >=15 and DiffCurVsTop5min < -5% return close otherwise Nan
    since = df.iloc[idx][sincePeakTime]
    if since and since >= 15 and df.iloc[idx][diffCurVsTop5min] < -0.05:
        setVal(df, idx, catchUpPrice, df.iloc[idx]['Close'])
    

## Route 1: caculate all indicators (close prices based)
for x in range(0, len(spy.index)):
    #print(spy.iloc[x])
    calCumDesc(spy, x), calPLessMA12(spy, x), calPeakTime(spy, x), calSincePeakTime(spy, x)
    calAvgTop5MinInLst36m(spy, x), calDiffCurVsTop5min(spy, x), calCatchUpPrice(spy, x)

# And.NTB Stock => NTB Stock Cost =? Catch Up Buy => Vlaue Target => TargetCumRequired
# 
def calCatchUpBuy(df, idx):
    if df.iloc[idx][catchUpPrice] and idx >= 5:
        catchup = max(df.iloc[idx-5:idx][ntbStockCost])
        setVal(df, idx, catchUpBuy, catchup if catchup > CATCHUP_INIT else CATCHUP_INIT)

def calValueTarget(df, idx):
    val = VAL_TARGET
    val += df.iloc[idx][catchUpBuy] if df.iloc[idx][catchUpBuy] else 0
    setVal(df, idx, valTarget, val)

def calTargetCumRequired(df, idx):
    val = df.iloc[idx][valTarget]
    val += df.iloc[idx-1][targetCumRequired] if idx > 0 else 0
    setVal(df, idx, targetCumRequired, val)

def calInStockNum(df, idx):
    if idx > 0:
        setVal(df, idx, inStockNum, df.iloc[idx-1][inventoryStockNum])
    
def calNeedToBuyStockNum(df, idx):
    row = df.iloc[idx]
    num = int(row[targetCumRequired] / row['Close']) - row[inStockNum]
    num = num if num > 0 else 0
    setVal(df, idx, needToBuyStockNum, num)

def calAdjNTBStockNum(df, idx):
    row = df.iloc[idx]
    needAdj = row[needToBuyStockNum] * row['Close'] > row[valTarget]
    needAdj = needAdj and (not row[cumDesc] or (idx > 0 and not df.iloc[idx-1][cumDesc]))
    val = int(row[valTarget]/row['Close']) if needAdj else row[needToBuyStockNum]
    setVal(df, idx, adjNTBStockNum, val)    
    setVal(df, idx, inventoryStockNum, row[inStockNum] + val)
    setVal(df, idx, ntbStockCost, row['Close'] * val)
    
## Route 2: caculate stocks number to buy
for x in range(0, len(spy.index)):
    calCatchUpBuy(spy, x), calValueTarget(spy, x), calTargetCumRequired(spy, x)
    calInStockNum(spy, x), calNeedToBuyStockNum(spy, x), calAdjNTBStockNum(spy, x)

spy.to_csv("SPY_output.csv")